In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [26]:
# !cp "drive/My Drive/Personal/Projects/Object Detection/libcudnn8_8.2.0.53-1+cuda11.3_amd64.deb" "/content"
!wget https://www.dropbox.com/s/0o63wwdkx4e75g2/libcudnn8_8.2.0.53-1%2Bcuda11.3_amd64.deb

--2021-11-13 20:08:03--  https://www.dropbox.com/s/0o63wwdkx4e75g2/libcudnn8_8.2.0.53-1%2Bcuda11.3_amd64.deb
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0o63wwdkx4e75g2/libcudnn8_8.2.0.53-1%2Bcuda11.3_amd64.deb [following]
--2021-11-13 20:08:06--  https://www.dropbox.com/s/raw/0o63wwdkx4e75g2/libcudnn8_8.2.0.53-1%2Bcuda11.3_amd64.deb
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7ab3e3c11df1856b77073c691f.dl.dropboxusercontent.com/cd/0/inline/BZ6vRiCJP-LeBdVOcE23UP0RsRTvKvIo2InEoI1_pQ6-3kps6gSXwZZNvOuH6vs47Aq0hlJguabAwkQQQsEc6bLYLhVNFChmn7G62hptkJs36f_-NhPWGn9TQyDh8GBD7ccVcouNf9fmvfeM_yUgPHTo/file# [following]
--2021-11-13 20:08:06--  https://uc7ab3e3c11df1856b77073c691f.dl.dropboxusercontent.com/cd/0/inline/BZ6

In [27]:
!dpkg -i "libcudnn8_8.2.0.53-1+cuda11.3_amd64.deb"
!ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so.8*

(Reading database ... 155220 files and directories currently installed.)
Preparing to unpack libcudnn8_8.2.0.53-1+cuda11.3_amd64.deb ...
Unpacking libcudnn8 (8.2.0.53-1+cuda11.3) over (8.2.0.53-1+cuda11.3) ...
Setting up libcudnn8 (8.2.0.53-1+cuda11.3) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

lrwxrwxrwx 1 root root     17 Apr 20  2021 /usr/lib/x86_64-linux-gnu/libcudnn.so.8 -> libcudnn.so.8.2.0
-rw-r--r-- 1 root root 162360 Apr 20  2021 /usr/lib/x86_64-linux-gnu/libcudnn.so.8.2.0


In [ ]:
!nvidia-smi

In [ ]:
!wget https://www.dropbox.com/s/bj742qjgnp3zhh2/Dataset.zip

In [2]:
!unzip -q "Dataset.zip"

**Restart Here**

In [3]:
import os 
import pathlib
from glob import glob 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%cd models/research
!pip install protobuf
!protoc object_detection/protos/*.proto --python_out=.
%cd ../..

In [ ]:
%cd models/research
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
%cd ../..

In [7]:
import numpy as np
import os
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import json
import pandas as pd
import glob
import os.path as osp
from pathlib import Path
import datetime
import random
import shutil
from io import StringIO, BytesIO
from PIL import Image
from IPython.display import display
import re

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
sns.set(rc={"font.size":9,"axes.titlesize":15,"axes.labelsize":9,
            "axes.titlepad":11, "axes.labelpad":9, "legend.fontsize":7,
            "legend.title_fontsize":7, 'axes.grid' : False})

from sklearn.model_selection import train_test_split

## Import object detection module
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils
from object_detection.protos.string_int_label_map_pb2 import StringIntLabelMap, StringIntLabelMapItem
from object_detection.utils import config_util
from object_detection.builders import model_builder

from google.protobuf import text_format

import tarfile
from numba import cuda 

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [8]:
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [9]:
pretrained_dir = "training_job/model/"

if not os.path.exists(pretrained_dir):
    os.makedirs(pretrained_dir)
    print('Pretrainined Model Directory:', pretrained_dir)

Pretrainined Model Directory: training_job/model/


In [10]:
## EfficientDet Configurations
MODELS_CONFIG = {
    'mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz',
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d4': {
        'model_name': 'efficientdet_d4_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d4_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d4_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d5': {
        'model_name': 'efficientdet_d5_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d5_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d5_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d6': {
        'model_name': 'efficientdet_d6_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d6_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d6_coco17_tpu-32.tar.gz',
    },
    'efficientdet-d7': {
        'model_name': 'efficientdet_d7_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d7_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d7_coco17_tpu-32.tar.gz',
    }
}

## Choosing D1 here for this tutorial
chosen_model = 'efficientdet-d2'
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [11]:
model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'+pretrained_checkpoint
# pretrained_dir = "/kaggle/working/models/research/object_detection/pretrained"

!wget {model_url}
!tar -xf {pretrained_checkpoint}
!mv {model_name}/ {pretrained_dir}
!rm {pretrained_checkpoint}

--2021-11-13 20:01:31--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d2_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.12.128, 2607:f8b0:400c:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.12.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62929273 (60M) [application/x-tar]
Saving to: ‘efficientdet_d2_coco17_tpu-32.tar.gz’

efficientdet_d2_coc 100%[===================>]  60.01M   168MB/s    in 0.4s    

2021-11-13 20:01:32 (168 MB/s) - ‘efficientdet_d2_coco17_tpu-32.tar.gz’ saved [62929273/62929273]



In [12]:
model_dir = os.path.join(pretrained_dir, model_name, "saved_model")

In [13]:
print("Pre-trained model directory", model_dir)
model = tf.saved_model.load(str(model_dir))

Pre-trained model directory training_job/model/efficientdet_d2_coco17_tpu-32/saved_model


In [14]:
def run_inference_for_single_image(model, image):
    '''
    Add a wrapper function to call the model, and cleanup the outputs:
    '''
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                  output_dict['detection_masks'], output_dict['detection_boxes'],
                   image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                           tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict

def show_inference(model, image_path):
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection.
    vis_utils.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

    return image_np
    
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path.

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np, boxes, classes, scores, category_index,
                    figsize=(12, 16), image_name=None):
    """Wrapper function to visualize detections.

    Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
    """
    image_np_with_annotations = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    else:
        plt.imshow(image_np_with_annotations)
        
        
def plot_img(img, size=(18, 18), is_rgb=True, title="", cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)

In [15]:
tfr_output_dir = "training_job/tfrecords/"

if not os.path.exists(tfr_output_dir):
    os.makedirs(tfr_output_dir)
    print('TFRecord Directory:', tfr_output_dir)

TFRecord Directory: training_job/tfrecords/


In [16]:
!wget https://raw.githubusercontent.com/hugozanini/object-detection/master/generate_tf_record.py

--2021-11-13 20:02:19--  https://raw.githubusercontent.com/hugozanini/object-detection/master/generate_tf_record.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5076 (5.0K) [text/plain]
Saving to: ‘generate_tf_record.py’

generate_tf_record. 100%[===================>]   4.96K  --.-KB/s    in 0s      

2021-11-13 20:02:19 (62.9 MB/s) - ‘generate_tf_record.py’ saved [5076/5076]



In [17]:
def convert_classes(classes, start=1):
    msg = StringIntLabelMap()
    for id, name in enumerate(classes, start=start):
        msg.item.append(StringIntLabelMapItem(id=id, name=name))
    text = str(text_format.MessageToBytes(msg, as_utf8=True), 'utf-8')
    return text

labels =  [
            "BoysShirt",
            "GirlsShirt",
            "BoysPant",
            "GirlsPant",
            "MenFootWear",
            "WomenFootWear"
            ]

txt = convert_classes(labels)
print(txt)
with open('labelmap.pbtxt', 'w') as f:
    f.write(txt)

item {
  name: "BoysShirt"
  id: 1
}
item {
  name: "GirlsShirt"
  id: 2
}
item {
  name: "BoysPant"
  id: 3
}
item {
  name: "GirlsPant"
  id: 4
}
item {
  name: "MenFootWear"
  id: 5
}
item {
  name: "WomenFootWear"
  id: 6
}



In [18]:
!python generate_tf_record.py -l labelmap.pbtxt -o training_job/tfrecords/train.record -i ShoppingDetectionData/Images -csv training.csv
!python generate_tf_record.py -l labelmap.pbtxt -o training_job/tfrecords/test.record -i ShoppingDetectionData/Images -csv testing.csv

INFO:Successfully created the TFRecords: training_job/tfrecords/train.record
INFO:Successfully created the TFRecords: training_job/tfrecords/test.record


In [19]:
model_files_dir = "training_job/model_files/"

if not os.path.exists(model_files_dir):
    os.makedirs(model_files_dir)
    print('Model Files Directory:', model_files_dir)

Model Files Directory: training_job/model_files/


In [20]:
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/'+base_pipeline_file
!wget {download_config} -P {model_files_dir}

--2021-11-13 20:02:27--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d2_768x768_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4632 (4.5K) [text/plain]
Saving to: ‘training_job/model_files/ssd_efficientdet_d2_768x768_coco17_tpu-8.config’

ssd_efficientdet_d2 100%[===================>]   4.52K  --.-KB/s    in 0s      

2021-11-13 20:02:28 (39.3 MB/s) - ‘training_job/model_files/ssd_efficientdet_d2_768x768_coco17_tpu-8.config’ saved [4632/4632]



In [21]:
## Get the Total Number of Classes

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

val_record_fname = 'training_job/tfrecords/test.record'
train_record_fname = 'training_job/tfrecords/train.record'
label_map_pbtxt_fname = 'labelmap.pbtxt'

pipeline_file = os.path.join(model_files_dir, base_pipeline_file)
fine_tune_checkpoint = os.path.join(pretrained_dir, model_name, 'checkpoint/ckpt-0')
num_classes = get_num_classes(label_map_pbtxt_fname)
print("No. of Classes", num_classes)
print("Checkpoint File Path", fine_tune_checkpoint)

No. of Classes 6
Checkpoint File Path training_job/model/efficientdet_d2_coco17_tpu-32/checkpoint/ckpt-0


In [22]:
## Training Configurations
num_epochs = 10
num_steps = 4000
num_eval_steps = 100
batch_size = 8 # Change this to 4 
print("Number of Steps:", num_steps)

Number of Steps: 4000


In [23]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

print('Updading the Training Configuration file')
with open(pipeline_file) as f:
    s = f.read()
    
with open(pipeline_file, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
               'input_path: "{}"'.format(train_record_fname), s)
    
    s = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")',
               'input_path: "{}"'.format(val_record_fname), s)

    # label_map_path
    s = re.sub('label_map_path: ".*?"',
               'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    
    #fine-tune checkpoint type
    s = re.sub('fine_tune_checkpoint_type: "classification"',
               'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

Updading the Training Configuration file


In [ ]:
## Check the pipeline config file
!cat {pipeline_file}

In [ ]:
model_dir = "training_job/model_files/training/"

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print('Training Files:', model_dir)

In [ ]:
del model
device = cuda.get_current_device()
device.reset()

In [ ]:
## Checking GPU Load
!nvidia-smi

Sat Nov 13 19:26:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    58W / 149W |      3MiB / 11441MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps} \
    --checkpoint_every_n=500

**You'll Stop Here**

In [ ]:
import glob
import os
os.environ['PYTHONPATH'] += ":models"
import sys
sys.path.append("models")

In [ ]:
model_dir = "training_job/model_files/training/"
pipeline_file = glob.glob(os.path.join('training_job/model_files/', '*.config'))[0]
pipeline_file

'training_job/model_files/ssd_efficientdet_d0_512x512_coco17_tpu-8.config'

In [ ]:
## Stop this cell manually
# !python models/research/object_detection/model_main_tf2.py \
#      --pipeline_config_path={pipeline_file} \
#      --model_dir={model_dir} \
#      --checkpoint_dir={model_dir} \
#      --run_once=True

In [ ]:
import numpy as np

import sys
import cv2
import glob
import os.path as osp
from pathlib import Path
import random
import io
import scipy.misc
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
sns.set(rc={"font.size":9,"axes.titlesize":15,"axes.labelsize":9,
            "axes.titlepad":11, "axes.labelpad":9, "legend.fontsize":7,
            "legend.title_fontsize":7, 'axes.grid' : False})

import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
model_dir = "training_job/model_files/training/"
pipeline_file = glob.glob(os.path.join('training_job/model_files/', '*.config'))[0]
pipeline_file

'training_job/model_files/ssd_efficientdet_d0_512x512_coco17_tpu-8.config'

In [ ]:
## See our saved weights
!ls {model_dir}

checkpoint		    ckpt-7.data-00000-of-00001
ckpt-3.data-00000-of-00001  ckpt-7.index
ckpt-3.index		    ckpt-8.data-00000-of-00001
ckpt-4.data-00000-of-00001  ckpt-8.index
ckpt-4.index		    ckpt-9.data-00000-of-00001
ckpt-5.data-00000-of-00001  ckpt-9.index
ckpt-5.index		    eval
ckpt-6.data-00000-of-00001  train
ckpt-6.index


In [ ]:
last_ckpt = Path(sorted(glob.glob(os.path.join(model_dir,'ckpt-*')))[-1]).stem
last_ckpt

In [ ]:
configs = config_util.get_configs_from_pipeline_file(pipeline_file)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

## Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(model_dir, last_ckpt))

def get_model_detection_function(model):
    """Get a tf.function for detection."""
    @tf.function
    def detect_fn(image):
        """Detect objects in image."""
        image, shapes = model.preprocess(image)
        prediction_dict = model.predict(image, shapes)
        detections = model.postprocess(prediction_dict, shapes)
        return detections, prediction_dict, tf.reshape(shapes, [-1])
    return detect_fn

# detect_fn = get_model_detection_function(detection_model)

In [ ]:
def plot_imgs(imgs, cols=2, size=10, is_rgb=True, title="", cmap='gray', img_size=None):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(round(cols*size*0.7), rows*size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)   
    
def draw_bbox(image, box, label, color):   
    alpha = 0.1
    alpha_box = 0.4
    overlay_bbox = image.copy()
    overlay_text = image.copy()
    output = image.copy()
    output = cv2.rectangle(output, (box[0], box[1]), (box[2], box[3]),
                           color, 2)

    text_width, text_height = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)[0]
    cv2.rectangle(overlay_bbox, (box[0], box[1]), (box[2], box[3]),
                  color, -1)
    cv2.addWeighted(overlay_bbox, alpha, output, 1 - alpha, 0, output)
    
    cv2.rectangle(overlay_text, (box[0], box[1]-7-text_height),
                  (box[0]+text_width+2, box[1]), (0, 0, 0), -1)
    
    cv2.addWeighted(overlay_text, alpha_box, output, 1 - alpha_box, 0, output)
    cv2.putText(output, label.upper(), (box[0], box[1]-5),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    return output

In [ ]:
output_directory = 'inference_graph'

!python models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

I1113 19:29:06.145456 140713424127872 ssd_efficientnet_bifpn_feature_extractor.py:143] EfficientDet EfficientNet backbone version: efficientnet-b0
I1113 19:29:06.145682 140713424127872 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet BiFPN num filters: 64
I1113 19:29:06.145804 140713424127872 ssd_efficientnet_bifpn_feature_extractor.py:146] EfficientDet BiFPN num iterations: 3
I1113 19:29:06.150083 140713424127872 efficientnet_model.py:147] round_filter input=32 output=32
I1113 19:29:06.176084 140713424127872 efficientnet_model.py:147] round_filter input=32 output=32
I1113 19:29:06.176253 140713424127872 efficientnet_model.py:147] round_filter input=16 output=16
I1113 19:29:06.259481 140713424127872 efficientnet_model.py:147] round_filter input=16 output=16
I1113 19:29:06.259777 140713424127872 efficientnet_model.py:147] round_filter input=24 output=24
I1113 19:29:06.457379 140713424127872 efficientnet_model.py:147] round_filter input=24 output=24
I1113 19:29:06.457574 140

In [ ]:
!zip -r saved_model.zip inference_graph/saved_model/

  adding: inference_graph/saved_model/ (stored 0%)
  adding: inference_graph/saved_model/saved_model.pb (deflated 93%)
  adding: inference_graph/saved_model/variables/ (stored 0%)
  adding: inference_graph/saved_model/variables/variables.data-00000-of-00001 (deflated 36%)
  adding: inference_graph/saved_model/variables/variables.index (deflated 78%)
  adding: inference_graph/saved_model/assets/ (stored 0%)


**Main : Donwload saved_model.zip**

In [ ]:
!wget https://raw.githubusercontent.com/hugozanini/object-detection/master/inferenceutils.py
from inferenceutils import *

--2021-11-13 19:31:21--  https://raw.githubusercontent.com/hugozanini/object-detection/master/inferenceutils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2219 (2.2K) [text/plain]
Saving to: ‘inferenceutils.py’

inferenceutils.py   100%[===================>]   2.17K  --.-KB/s    in 0s      

2021-11-13 19:31:21 (27.0 MB/s) - ‘inferenceutils.py’ saved [2219/2219]



In [ ]:
output_directory = 'inference_graph/'

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap("/content/labelmap.pbtxt", use_display_name=True)
tf.keras.backend.clear_session()
model = tf.saved_model.load(f'/content/{output_directory}/saved_model')

In [ ]:
import pandas as pd
test = pd.read_csv('/content/testing.csv')
images = list(test['filename'][0:20])

In [ ]:
outputs = test['class'].values

In [ ]:
# images = ['canguru-1.jpeg', 'canguru-2.jpeg', 'canguru-3.jpeg']

for e, image_name in enumerate(images):
  print(outputs[e])
  image_np = load_image_into_numpy_array('/content/ShoppingDetectionData/Images/' + image_name)
  output_dict = run_inference_for_single_image(model, image_np)
  # print(output_dict)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
  display(Image.fromarray(image_np))

Output hidden; open in https://colab.research.google.com to view.